In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import math
import scipy.ndimage
import PIL.Image
import os
import glob

In [ ]:
def snn_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape

    # equalizing the histogram
    '''''val1,val2 = np.percentile(img_array, (2,90))
    new_img= exposure.rescale_intensity(img_array, in_range=(val1,val2))
    plt.figure(figsize=(7, 7))                   
    fig = plt.imshow(arr_new, cmap = 'gray')  
    plt.axis(False)'''''
    
    eq=exposure.equalize_hist(img_array).astype(np.float32)
    
    # define filter function
    
    def snn(image, window_size=5):

        def nearest(a, num):
            return a.flat[np.abs(a - num).argmin()]  
    
        def func(kernel, pairs):
            centre_pixel = kernel[kernel.size // 2]
            select = [nearest(kernel[p], centre_pixel) for p in pairs]
            return np.mean(select)

        pairs = [[i, window_size**2-1 - i] for i in range(window_size**2 // 2)]
        return scipy.ndimage.generic_filter(image, func, size= window_size,
                                        extra_keywords={'pairs': pairs}
                                       )
    #filter image
    
    filtered_image= snn(img_array)
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_SNN.tif'
    output_file = output_file[:-2] + ['SNN'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    snn_filter(input_file, output_file)